In [ ]:
library(dplyr)
library(data.table)
library(tidyverse)
library(ggplot2)
library(ggalt)
library(viridis)
library(patchwork)
library(lubridate)
library(ggh4x)
library(ggrepel)

# Analysis on pairwise distance

In [ ]:
df <- fread(snakemake@input$stabilities_pairwise) %>%
    mutate(dM = (Mean_Lineage - Mean_Ref_Lineage)/Mean_Ref_Lineage*100)
names(df)

In [ ]:
dfsort <- df %>%
    select(Lineage,dM) %>%
    arrange(-dM)
df$Lineage <- factor(df$Lineage,levels = dfsort$Lineage)
df <- df %>%
    mutate(Wilcox_less_P_sig = if_else(Wilcox_less_P <0.05, 1,0))
df

In [ ]:
options(repr.plot.width=9, repr.plot.height=4, jupyter.plot_mimetypes = "image/svg+xml")
p1 <- ggplot(data=df) +geom_lollipop(aes(x=Lineage,y=dM,color=Lineage),point.size=4) + guides(colour = "none") +
ylab("Stability change, %") + theme(text = element_text(size = 20),axis.title.y = element_text(size = 14)) + geom_hline(yintercept = 0, color="red")

p1

In [ ]:
dfP <- df %>%
    select(Lineage, ends_with("P")) %>%
    pivot_longer(cols = ends_with("P"),values_to = "Values", names_to = "Measures") %>%
    mutate(Sig = if_else(Values < 0.05,0,1)) %>%
    filter(grepl("zinb",Measures) | grepl("Wil",Measures))
p2 <- ggplot(data = dfP) + geom_label(aes(x=Lineage, y=1,color=factor(Sig),label=format(round(Values,digits = 2), nsmall = 2  ) ) ) + facet_grid(vars(Measures)) +
guides(colour = "none") +
theme_bw() + theme(panel.border = element_blank(), panel.grid.major = element_blank(),
panel.grid.minor = element_blank(), axis.line = element_line(colour = "white"),strip.text.y.right = element_text(angle = 0, size=12, face="bold"),
        axis.title.x=element_blank(),
        axis.text.x=element_blank(),
        axis.ticks.x=element_blank(),
        axis.title.y=element_blank(),
        axis.text.y=element_blank(),
        axis.ticks.y=element_blank(),
) +
ylab("") + xlab("")
df